# Sepic Converter [5 V, 20 V] -> 10 V
This is the SEPIC converter for converting voltages for the level-shifters of the FET-drivers.

### Why SEPIC?
- Output only goes above 0 V when the switch is turned on (a.k.a. when sepic starts working)
- Less noise on power line (isolated supply through secondary inductor)
- 10 V level shifting working both at 5 V or at 20 V supply (20 V => 40 V)

## Inductor sizing
We can choose 2 inductors wound around the same common core. This would 
- Give us more boardspace
- Require smaller inductors in terms of magnetization (half as big)

This option seems to be more expensive however, so think about simply buying twice the inductor, since the board-size is not a limitation anyways.

### Variables required for calculation

In [ ]:

#############! DESIGN PARAMETERS ##################

I_totrms_A = 5

### Take from LTSpice simulation + add on top
I_rms_A = 0.5 # Amps (slightly overdimensioned already)
I_peak_A = 1 # Amps, for each bridge spike of 1 amp 180 degrees shifted

### DIODE Choice:
# 40 V max
# 1 A RMS current max
V_diode = 0.55 # SS14
V_max_diode = 20 # 

### MOSFET Choice: AO3400A-mosfet
Vfet_gsth = 1.45 # Volt
Vfet_diode = 1 # Volt
Rfet_dsn = 50e-3 # At 3 amps, 2.5 V
Vfet_dsn = Rfet_dsn * I_totrms_A
print(f"Vfet_dsn {Vfet_dsn}")

# Required Output Voltage
Vout = 10

Vfet_dsn 0.25


### Case 1:  (Vin=5 V) lowest possible input voltage

In [190]:

#! >>> SET INPUT VOLTAGE min max
Vin_list = [20, 5]
#! <<< SET INPUT VOLTAGE

In [191]:
for Vin in Vin_list:
	print(f"\r\n##################################################")
	print(f"################# VIN == {Vin} ###################")
	print(f"##################################################")

	D_sepic = (Vout + V_diode) / (Vout + Vin - Rfet_dsn * I_totrms_A  + V_diode)
	print(f"Duty cycle: {D_sepic}, time the switch is closed")

	if (D_sepic > 0.5):
		print(f"Warning slope compensation required")

	##### MINIMUM INDUCTOR SIZE #######
	print(f"\r\nMINIMUM INDUCTOR SIZE")
	freq_sw = 500e3

	I1_sepic = D_sepic * I_rms_A / (1-D_sepic)
	I2_sepic = I_rms_A

	# Inductance
	L1_sepic = (Vin - Vfet_dsn) * (1-D_sepic) / (2 * I_rms_A * freq_sw)
	L2_sepic = (Vin - Vfet_dsn) * (D_sepic) / (2 * I_rms_A * freq_sw)

	print(f"L1_sepic: {L1_sepic*1e6} uH, L2_sepic: {L2_sepic*1e6} uH")


##################################################
################# VIN == 20 ###################
##################################################
Duty cycle: 0.3481848184818482, time the switch is closed

MINIMUM INDUCTOR SIZE
L1_sepic: 25.746699669967 uH, L2_sepic: 13.753300330033003 uH

##################################################
################# VIN == 5 ###################
##################################################
Duty cycle: 0.6895424836601307, time the switch is closed
Warning slope compensation required

MINIMUM INDUCTOR SIZE
L1_sepic: 2.9493464052287583 uH, L2_sepic: 6.550653594771242 uH


### SEPIC passives sizing
#### 
#### Required slope compensation 
Check **[dcdc_choice.md](../dcdc_choice.md)** for more information about current sensing slope.

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#! >>> INSERT CHOICE
# Set ESR of output capacitor
ESR_cout = 0.1 # ESR of output capacitor
L1_ch = 60e-6#L1_sepic
L2_ch = 60e-6#L2_sepic
#! <<< INSERT CHOICE
import math 
for Vin in Vin_list:
	print(f"\r\n##################################################")
	print(f"################### VIN == {Vin} #####################")
	print(f"##################################################")

	D_sepic = (Vout + V_diode) / (Vout + Vin - Rfet_dsn * I_totrms_A  + V_diode)
	print(f"Duty cycle: {D_sepic}, time the switch is closed")

	if (D_sepic > 0.5):
		print(f"Warning slope compensation required")

	##### MINIMUM INDUCTOR SIZE #######
	print(f"\r\nMINIMUM INDUCTOR SIZE")
	freq_sw = 500e3

	I1_sepic = D_sepic * I_rms_A / (1-D_sepic)
	I2_sepic = I_rms_A

	# Inductance
	L1_sepic = (Vin - Vfet_dsn) * (1-D_sepic) / (2 * I_rms_A * freq_sw)
	L2_sepic = (Vin - Vfet_dsn) * (D_sepic) / (2 * I_rms_A * freq_sw)

	print(f"L1_sepic: {L1_sepic*1e6} uH, L2_sepic: {L2_sepic*1e6} uH")
	

	##### PEAK CURRENT CALCULATION #######
	print(f"\r\n#### PEAK CURRENT CALCULATION ####")
	# Peak current
	dIl1 = (Vin - Vfet_dsn) * D_sepic / (L1_ch * freq_sw)
	dIl2 = (Vin - Vfet_dsn) * D_sepic / (L2_ch * freq_sw)
	print(f"dIl1: {dIl1} A, dIl2: {dIl2} A")

	Il1_peak = I1_sepic + dIl1 / 2
	Il2_peak = I2_sepic + dIl2 / 2
	print(f"Il1_peak: {Il1_peak} A, Il2_peak: {Il2_peak} A")

	######## OUTPUT VOLTAGE RIPPLE #########
	print(f"\r\n#### OUTPUT VOLTAGE RIPPLE (ESR-DEPENDENT) ####")

	dVout = ((I_rms_A / (1-D_sepic)) + dIl2 / 2) * ESR_cout
	print(f"dVout: {dVout} V")

	######## SEPIC CAPACITOR SELECTION #########
	print(f"\r\n#### SEPIC CAPACITOR SELECTION ####")
	I_sw_peak = I1_sepic + I_rms_A + (dIl1 + dIl2) / 2
	I_swrms = math.sqrt((I_sw_peak**2 - I_sw_peak*(dIl1 + dIl2) + (dIl1 + dIl2)**2 / 3)*D_sepic)
	print(f"I_SW_RMS: {I_swrms} A")
	print(f"I_SW_PEAK: {I_sw_peak} A")

	I_cap_rms = math.sqrt(I_swrms**2 + (Il1_peak**2-Il1_peak*dIl1+dIl1**2) * (1-D_sepic))
	V_cap_rms = Vin
	C_min = L1_ch * I_rms_A**2 / (Vin-Vfet_dsn)**2

	print(f"I_cap_rms {I_cap_rms} A, V_max: {V_cap_rms} V, cap_min {C_min*1e6} uF")

	######## INPUT CAPACITOR SELECTION #########
	print(f"\r\n#### INPUT CAPACITOR SELECTION ####")
	I_cin_rms = (D_sepic / (2 * math.sqrt(3))) * ((Vin - Vfet_dsn) / (L1_ch * freq_sw))
	print(f"Input capacitor RMS current {I_cin_rms} A")


	######## OUTPUT CAPACITOR SELECTION #########
	print(f"\r\n#### OUTPUT CAPACITOR SELECTION ####")
	I_cout_rms = math.sqrt((I_sw_peak**2 - I_sw_peak * (dIl1 + dIl2) + ((dIl2+ dIl1)**2)/3.0) * (1-D_sepic) - I_rms_A**2)
	print(f"output capacitor RMS current: {I_cout_rms} A")


	######## SLOPE COMPENSATION #######
	print(f"\r\n#### SLOPE COMPENSATION, SENSE RESISTOR SELECTION ####")
	V_sense = 0.156 # Volt - FIXED current sense threshold voltage
	V_sl = 92e-3 # Internal compensation ramp voltage
	V_sl_ratio = 0.49
	R_sl = 1
	K_sl = 40e-6
	dV_sl = K_sl * R_sl

	### R_sen estimation (boost equations)
	R_sense = (V_sense - D_sepic * (V_sl + dV_sl)) / I_sw_peak # (eq. 48)
	print(f"R_sense: {R_sense} Ohm")
	print(f"I_max: {V_sense / R_sense} A")
	### Slope compensation
	# R_sl = ((R_sense * (Vout - 2*Vin)) / (2*freq_sw*L1_ch) - V_sl) / K_sl
	print(f"Slope compensation resistor: {R_sl}, Duty: {D_sepic}")


##################################################
################### VIN == 20 #####################
##################################################
Duty cycle: 0.3481848184818482, time the switch is closed

MINIMUM INDUCTOR SIZE
L1_sepic: 25.746699669967 uH, L2_sepic: 13.753300330033003 uH

#### PEAK CURRENT CALCULATION ####
dIl1: 0.22922167216721673 A, dIl2: 0.22922167216721673 A
Il1_peak: 0.381699443678545 A, Il2_peak: 0.6146108360836083 A

#### OUTPUT VOLTAGE RIPPLE (ESR-DEPENDENT) ####
dVout: 0.08816994436785451 V

#### SEPIC CAPACITOR SELECTION ####
I_SW_RMS: 0.45932429039814143 A
I_SW_PEAK: 0.9963102797621535 A
I_cap_rms 0.5321305657150515 A, V_max: 20 V, cap_min 0.03845537574106714 uF

#### INPUT CAPACITOR SELECTION ####
Input capacitor RMS current 0.06617059706491936 A

#### OUTPUT CAPACITOR SELECTION ####
output capacitor RMS current: 0.3807365490938508 A

#### SLOPE COMPENSATION, SENSE RESISTOR SELECTION ####
R_sense: 0.12441211520624046 Ohm
I_max: 1.253897176664794 A


## Setting output voltage
- Don't forget to connect an output capacitor between feedback and ground pins
	- Reduces bandwidth
	- Reduces susceptibility to noise

In [193]:
RF1 = 60e3 # 60 kOhms feedback in example 
RF2 = (1.26 * RF1) / (Vout - 1.26)

print(f"Resistors: {RF1*1e-3:.2f} kOhm, {RF2*1e-3:.2f} kOhm")

Resistors: 60.00 kOhm, 8.65 kOhm


## Setting Current Limit

### Compensation design for RHP in voltage feedback loop
(Check TI-AN1484)
The right-half plane zero:
- Shifts the phase -90 degrees, so make sure crossover happens way before f_rhzp
- Is lowest and thus more critical with D highest, so choose V = 5 V (lowest) since D is highest there.

The resonant LC pole:
- Make sure the bandwidth is far below that, at resonance we get a phase-shift of -180 degrees

Crossover frequency:
- Frequency where open-loop transfer function drops below 0 dB (Gain: 1)

Make sure to lower cutoff frequency so positive feedback is avoided.

In [200]:

#! >>> INSERT COUPLING CAPACITOR CHOICE HERE
C_coupling_ch = 10e-6 # F
D_sepic_max = 0.65
Vin = 5
Cin_ch = 100e-6
Cout_ch = 100e-6
#! <<< INSERT COUPLING CAPACITOR CHOICE HERE

# LOAD POLE ESTIMATION
print(f"\r\n##################################################")
print(f"################### VIN == {Vin} #####################")
print(f"##################################################")
# RIGHT HALF PLANE ZERO
f_RHPZ = (1 - D_sepic_max) **2 * Vout / (2 * math.pi * D_sepic_max * L2_ch * 0.5 * I_rms_A)
print(f"Right Half plane zero: {f_RHPZ*1e-3:.2f}kHz")
# RESONANT LC POLE 
f_R = 1 / (2*math.pi * math.sqrt(L2_ch * C_coupling_ch))
print(f"Cs-L2 Resonance: {f_R*1e-3:.2f}kHz")
f_crossover = min(f_R, f_RHPZ) / 6
print(f"Crossover frequency: {f_crossover*1e-3:.2f}kHz")

#### COMPENSATION NETWORK CHOICE
# CHOICE OF RC
Gcs = 1/R_sense # Current sense gain
print(f"Gcs: {Gcs}")
Gma = 800e-6 # Error Amplifier Transconductance
Rc = (2 * math.pi * f_crossover * Cout_ch * Vout**2 * (1+D_sepic_max))
Rc = Rc / (Gcs*Gma*1.26*Vin*D_sepic_max)
print(f"Rc: {Rc*1e-3:.2f}kOhm")

# CHOICE OF CC1 (1/4th crossover frequency)
Cc1 = 4 / (2 * math.pi * f_crossover * Rc)
Cc2 = Cout_ch * ESR_cout / Rc
print(f"Cc1: {Cc1*1e9}nF, Cc2: {Cc2*1e9}nF")


##################################################
################### VIN == 5 #####################
##################################################
Right Half plane zero: 20.00kHz
Cs-L2 Resonance: 6.50kHz
Crossover frequency: 1.08kHz
Gcs: 11.023742792963638
Rc: 3.11kOhm
Cc1: 189.10489164086692nF, Cc2: 3.216739530447416nF


## Mosfet / Diode Selection

In [ ]:
V_sw_peak = Vin + Vout + V_diode
print(f"V_sw_peak: {V_sw_peak}")

V_sw_peak: 30.55
